In [1]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import requests
from io import StringIO
from os import listdir
from os.path import isfile, join
import datetime as dt
from datetime import datetime, timedelta

In [2]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f



In [3]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"

In [4]:
ISOS = ["NYISO"]

In [27]:
ISO_files = {}
ISO_data = {}
ISO_final = {}

#load in
for ISO in ISOS:
    ISO_files[ISO] = listdir_nohidden(f"{FILE_PATH}/{ISO}/old")
    ISO_data[ISO] = {}
    for file in tqdm(ISO_files[ISO]):
        compiled = re.compile(re.escape(f"{ISO}_"), re.IGNORECASE)

        ISO_data[ISO][compiled.sub("", file).replace(".csv","")] = pd.read_csv(f"{FILE_PATH}/{ISO}/old/{file}")

14it [00:01,  7.76it/s]


In [28]:
#add merge datetime
titles = {}
for key in ISO_data:
    print(key)
    titles[key]= []
    for k in ISO_data[key]:
        try:
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
        except:
            ISO_data[key][k] = ISO_data[key][k][ISO_data[key][k].index % 24 != 0]
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
            print(type(ISO_data[key][k]["time_tomerge"]))

NYISO


In [31]:
ISO_final = {}
def clean_final(ISO: str, df: str):
    ISO_data[ISO][df].columns = ISO_data[ISO][df].columns.str.lower()
    no_date = [col for col in ISO_data[ISO][df].columns if "date" not in col]
    ISO_data[ISO][df] = ISO_data[ISO][df][no_date]
    try:
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("unnamed: 0")
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("hour")
    except:
        pass
    ISO_data[ISO][df] = ISO_data[ISO][df].sort_values(by="time_tomerge")
    #ISO_data[ISO][df] = ISO_data[ISO][df].set_index("time_tomerge")
    ISO_final[ISO] = ISO_data[ISO][df]
    for column in ISO_final[ISO].columns:
        if str(ISO).lower() in column:
            ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
    print(ISO_final[ISO])

In [32]:
for iso in ISO:
    clean_final(ISO, "all_data")

                            da_10sr  da_10nsr  da_30or  da_regcap  rt_10sr  \
time_tomerge                                                                 
2014-01-01 05:00:00+00:00  3.640000      1.18     0.54       7.00      0.0   
2014-01-01 06:00:00+00:00  3.640000      1.18     0.54       7.00      0.0   
2014-01-01 07:00:00+00:00  0.900000      0.90     0.54       7.00      0.0   
2014-01-01 08:00:00+00:00  0.900000      0.90     0.54       4.00      0.0   
2014-01-01 09:00:00+00:00  0.900000      0.90     0.54       7.00      0.0   
...                             ...       ...      ...        ...      ...   
2019-01-01 00:00:00+00:00  5.976364      5.00     5.00      10.36      0.0   
2019-01-01 01:00:00+00:00  3.500000      3.50     3.50       5.00      0.0   
2019-01-01 02:00:00+00:00  3.000000      3.00     3.00      12.00      0.0   
2019-01-01 03:00:00+00:00  3.000000      3.00     3.00      13.08      0.0   
2019-01-01 04:00:00+00:00  3.000000      3.00     3.00      21.1

In [33]:
ISO_final["NYISO"] =ISO_final["NYISO"].reset_index()
ISO_final["NYISO"]["time_tomerge"] = ISO_final["NYISO"]["time_tomerge"] - timedelta(hours=5)
print(ISO_final["NYISO"].head())

               time_tomerge  da_10sr  da_10nsr  da_30or  da_regcap  rt_10sr  \
0 2014-01-01 00:00:00+00:00     3.64      1.18     0.54        7.0      0.0   
1 2014-01-01 01:00:00+00:00     3.64      1.18     0.54        7.0      0.0   
2 2014-01-01 02:00:00+00:00     0.90      0.90     0.54        7.0      0.0   
3 2014-01-01 03:00:00+00:00     0.90      0.90     0.54        4.0      0.0   
4 2014-01-01 04:00:00+00:00     0.90      0.90     0.54        7.0      0.0   

   rt_10nsr  rt_30or  rt_regcap  rt_regmov  ...  gen_hydro  gen_gas  \
0       0.0      0.0   7.000000     0.1500  ...        NaN      NaN   
1       0.0      0.0   6.116667     0.0250  ...        NaN      NaN   
2       0.0      0.0   5.616667     0.0350  ...        NaN      NaN   
3       0.0      0.0   4.635000     0.0000  ...        NaN      NaN   
4       0.0      0.0   5.705000     0.0475  ...        NaN      NaN   

   gen_nuclear  gen_other_fossil  gen_other_renewables gen_wind     load  \
0          NaN        

In [34]:
ISO_done = ["NYISO"]
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"
new_data = {}
for ISO in ISO_done:
    sub_folder = f"{FILE_PATH}/{ISO}/new/combined"
    files = [f for f in listdir(sub_folder) if isfile(join(sub_folder, f)) and f[0]!="."]
    new_data[ISO] = {}
    print(ISO)
    for file in tqdm(files):
        new_data[ISO][file.replace(".csv", "")] = pd.read_csv(f"{sub_folder}/{file}")

NYISO


100%|██████████| 8/8 [00:25<00:00,  3.13s/it]


In [35]:
new_data[ISO]

{'ancillary_da':         Unnamed: 0    Eastern Date Hour Eastern Date Hour Time Zone Zone Name  \
 0                0  2019/01/01 00:00:00                         EST    CAPITL   
 1               10  2019/01/01 00:00:00                         EST      WEST   
 2                9  2019/01/01 00:00:00                         EST     NORTH   
 3                8  2019/01/01 00:00:00                         EST    N.Y.C.   
 4                7  2019/01/01 00:00:00                         EST    MILLWD   
 ...            ...                  ...                         ...       ...   
 377779       48581  2022/12/31 23:00:00                         EST    LONGIL   
 377780       48582  2022/12/31 23:00:00                         EST    MHK VL   
 377781       48583  2022/12/31 23:00:00                         EST    MILLWD   
 377782       48584  2022/12/31 23:00:00                         EST    N.Y.C.   
 377783       48586  2022/12/31 23:00:00                         EST      WEST   


In [36]:
[key for key in new_data[ISO]]

['ancillary_da',
 'energy_da',
 'ancillary_rt',
 'energy_rt',
 'load_forecast',
 'generation',
 'load',
 'exchange']

In [37]:
[key for key in ISO_final[ISO]]

['time_tomerge',
 'da_10sr',
 'da_10nsr',
 'da_30or',
 'da_regcap',
 'rt_10sr',
 'rt_10nsr',
 'rt_30or',
 'rt_regcap',
 'rt_regmov',
 'congestion_da',
 'congestion_rt',
 'energy_da',
 'energy_rt',
 'exchange',
 'unnamed: 0',
 'gas_price',
 'gen_dual_fuel',
 'gen_hydro',
 'gen_gas',
 'gen_nuclear',
 'gen_other_fossil',
 'gen_other_renewables',
 'gen_wind',
 'load',
 'load_forecast',
 'losses_da',
 'losses_rt']

In [38]:
#ancillary
print(np.unique(new_data[ISO]["ancillary_rt"].columns))
print(new_data[ISO]["ancillary_rt"].head())

['RTD 10 Min Non Sync' 'RTD 10 Min Sync' 'RTD 30 Min Non Sync'
 'RTD AS Price Version' 'RTD End Time Stamp'
 'RTD NYCA Regulation Capacity' 'RTD NYCA Regulation Movement'
 'Unnamed: 0' 'Zone Name' 'Zone PTID' 'time_tomerge']
   Unnamed: 0   RTD End Time Stamp Zone Name  Zone PTID  RTD 10 Min Sync  \
0           0  2019/01/01 00:05:00    CAPITL      61757              0.0   
1          10  2019/01/01 00:05:00      WEST      61752              0.0   
2           9  2019/01/01 00:05:00     NORTH      61755              0.0   
3           8  2019/01/01 00:05:00    N.Y.C.      61761              0.0   
4           7  2019/01/01 00:05:00    MILLWD      61759              0.0   

   RTD 10 Min Non Sync  RTD 30 Min Non Sync  RTD NYCA Regulation Capacity  \
0                  0.0                  0.0                         37.74   
1                  0.0                  0.0                         37.74   
2                  0.0                  0.0                         37.74   
3         

In [39]:
new_data[ISO]['ancillary_da']["time_tomerge"] = pd.to_datetime(new_data[ISO]['ancillary_da']["time_tomerge"], utc = True)
#group by time and average all variables
ancillary_da = pd.DataFrame(new_data[ISO]['ancillary_da'].groupby(['time_tomerge'])['DAM 10 Min Non Sync', 'DAM 10 Min Sync', 'DAM 30 Min Non Sync', 'DAM NYCA Regulation Capacity'].mean())

#rename
ancillary_da = ancillary_da.rename(columns = {'DAM 10 Min Non Sync': 'da_10nsr', 'DAM 10 Min Sync': 'da_10sr', 'DAM 30 Min Non Sync': "da_30or", 'DAM NYCA Regulation Capacity': 'da_regcap'})
print(ancillary_da)

                           da_10nsr   da_10sr   da_30or  da_regcap
time_tomerge                                                      
2019-01-01 00:00:00+00:00  3.000000  3.000000  3.000000      26.77
2019-01-01 01:00:00+00:00  3.000000  3.000000  3.000000      21.10
2019-01-01 02:00:00+00:00  3.000000  3.000000  3.000000      21.10
2019-01-01 03:00:00+00:00  3.000000  3.000000  3.000000       3.00
2019-01-01 04:00:00+00:00  3.000000  3.000000  3.000000      21.10
...                             ...       ...       ...        ...
2022-12-31 19:00:00+00:00  6.074545  6.074545  4.983636      15.21
2022-12-31 20:00:00+00:00  5.381818  5.381818  4.700000       5.95
2022-12-31 21:00:00+00:00  5.381818  5.381818  4.700000       8.87
2022-12-31 22:00:00+00:00  6.422727  6.422727  5.850000       6.90
2022-12-31 23:00:00+00:00  6.545455  6.545455  6.000000      12.90

[34340 rows x 4 columns]


/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_4194/3274709947.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ancillary_da = pd.DataFrame(new_data[ISO]['ancillary_da'].groupby(['time_tomerge'])['DAM 10 Min Non Sync', 'DAM 10 Min Sync', 'DAM 30 Min Non Sync', 'DAM NYCA Regulation Capacity'].mean())


In [40]:
#repeat for rt
new_data[ISO]['ancillary_rt']["time_tomerge"] = pd.to_datetime(new_data[ISO]['ancillary_rt']["time_tomerge"], utc = True).dt.floor("H")
ancillary_rt = pd.DataFrame(new_data[ISO]['ancillary_rt'].groupby(['time_tomerge'])['RTD 10 Min Non Sync', 'RTD 10 Min Sync', 'RTD 30 Min Non Sync', 'RTD NYCA Regulation Capacity', 'RTD NYCA Regulation Movement'].mean())
ancillary_rt = ancillary_rt.rename(columns = {'RTD 10 Min Non Sync': 'rt_10nsr', 'RTD 10 Min Sync': 'rt_10sr', 'RTD 30 Min Non Sync': "rt_30or", 'RTD NYCA Regulation Capacity': 'rt_regcap', 'RTD NYCA Regulation Movement': 'rt_regmov'})
print(ancillary_rt)

/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_4194/15502459.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ancillary_rt = pd.DataFrame(new_data[ISO]['ancillary_rt'].groupby(['time_tomerge'])['RTD 10 Min Non Sync', 'RTD 10 Min Sync', 'RTD 30 Min Non Sync', 'RTD NYCA Regulation Capacity', 'RTD NYCA Regulation Movement'].mean())


                           rt_10nsr   rt_10sr  rt_30or  rt_regcap  rt_regmov
time_tomerge                                                                
2019-01-01 00:00:00+00:00       0.0  2.825035      0.0  26.334615   0.250000
2019-01-01 01:00:00+00:00       0.0  0.000000      0.0  21.100000   0.300000
2019-01-01 02:00:00+00:00       0.0  0.000000      0.0  21.100000   0.300000
2019-01-01 03:00:00+00:00       0.0  0.000000      0.0   0.000000   0.300000
2019-01-01 04:00:00+00:00       0.0  0.000000      0.0  17.583333   0.300000
...                             ...       ...      ...        ...        ...
2022-12-31 20:00:00+00:00       0.0  0.000000      0.0   5.058333   0.154167
2022-12-31 21:00:00+00:00       0.0  0.000000      0.0  14.181667   0.361667
2022-12-31 22:00:00+00:00       0.0  0.000000      0.0   7.751667   0.191667
2022-12-31 23:00:00+00:00       0.0  0.000000      0.0   8.290000   0.109167
2023-01-01 00:00:00+00:00       0.0  0.000000      0.0  20.700000   0.060000

In [41]:
l1 = list(ancillary_da.index)
l2 = list(ancillary_rt.index)
missing = []
for l in tqdm(range(len(l2))):
    if l2[l] not in l1:
        missing.append(l2[l])
missing

##rt is missing dec 2021 data

100%|██████████| 35061/35061 [00:42<00:00, 823.69it/s] 


[Timestamp('2021-12-02 00:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 01:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 02:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 03:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 04:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 05:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 06:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 07:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 08:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 09:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 10:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 11:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 12:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 13:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 14:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 15:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 16:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 17:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 18:00:00+0000', tz='UTC'),
 Timestamp('2021-12-02 19:00:00+0000', tz='UTC'),


In [42]:
#new_data[ISO]["energy_rt"] = new_data[ISO]["energy_rt"].drop(columns=["Unnamed: 0", 'Unnamed: 0.1'])
#convert to utc and then back to EST
new_data[ISO]["energy_rt"]["time_tomerge"] = pd.to_datetime(new_data[ISO]["energy_rt"]["Time"], utc=True)-timedelta(hours=5)
new_data[ISO]["energy_rt"] = new_data[ISO]["energy_rt"].sort_values(by="time_tomerge")

In [43]:
#average, rename and remove 2018
subset = new_data[ISO]["energy_rt"]
subset["time_tomerge"] = subset["time_tomerge"].dt.floor("H")
subset = pd.DataFrame(subset.groupby(["time_tomerge"])["Energy", "Congestion", "Loss"].mean())
subset = subset.drop_duplicates().rename(columns={'Energy':'energy_rt','Congestion':'congestion_rt', "Loss": "losses_rt"})
print(subset[subset.index.year !=2018])
energy_rt = subset[subset.index.year !=2018]

/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_4194/1061079319.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  subset = pd.DataFrame(subset.groupby(["time_tomerge"])["Energy", "Congestion", "Loss"].mean())


                           energy_rt  congestion_rt  losses_rt
time_tomerge                                                  
2019-01-01 00:00:00+00:00   1.957095     -17.284857   0.056524
2019-01-01 01:00:00+00:00  10.012556      -5.570167   0.275111
2019-01-01 02:00:00+00:00  13.475222      -3.718444   0.370056
2019-01-01 03:00:00+00:00   1.196444      -8.981056   0.036500
2019-01-01 04:00:00+00:00   5.549444      -1.556167   0.156111
...                              ...            ...        ...
2022-12-31 20:00:00+00:00  30.349611       0.816389   0.801000
2022-12-31 21:00:00+00:00  25.640444       0.538000   0.609333
2022-12-31 22:00:00+00:00  26.697000       0.000389   0.645944
2022-12-31 23:00:00+00:00  25.833222      -0.984333   0.647000
2023-01-01 00:00:00+00:00  26.270667      -5.906000   0.651333

[35065 rows x 3 columns]


In [44]:
#repeat for da
#new_data[ISO]["energy_rt"] = new_data[ISO]["energy_rt"].drop(columns=["Unnamed: 0", 'Unnamed: 0.1'])
new_data[ISO]["energy_da"]["time_tomerge"] = pd.to_datetime(new_data[ISO]["energy_da"]["Time"], utc=True)-timedelta(hours=5)
new_data[ISO]["energy_da"] = new_data[ISO]["energy_da"].sort_values(by="time_tomerge")

In [45]:
subset = new_data[ISO]["energy_da"]
subset["time_tomerge"] = subset["time_tomerge"].dt.floor("H")
subset = pd.DataFrame(subset.groupby(["time_tomerge"])["Energy", "Congestion", "Loss"].mean())
subset = subset.drop_duplicates().rename(columns={'Energy':'energy_da','Congestion':'congestion_da', "Loss": "losses_da"})
print(subset[subset.index.year !=2018])
energy_da =subset[subset.index.year !=2018]

                           energy_da  congestion_da  losses_da
time_tomerge                                                  
2019-01-01 00:00:00+00:00  10.338000      -7.284000   0.332000
2019-01-01 01:00:00+00:00  10.318000      -5.680000   0.311333
2019-01-01 02:00:00+00:00   9.527333      -4.806000   0.290667
2019-01-01 03:00:00+00:00   9.544000      -4.700667   0.287333
2019-01-01 04:00:00+00:00   9.432667      -4.912667   0.292667
...                              ...            ...        ...
2022-12-31 19:00:00+00:00  32.485333      -5.516667   0.883333
2022-12-31 20:00:00+00:00  32.455333      -2.342667   0.886000
2022-12-31 21:00:00+00:00  30.455333      -2.824000   0.740667
2022-12-31 22:00:00+00:00  28.338000      -2.798667   0.720667
2022-12-31 23:00:00+00:00  21.406667      -8.022000   0.548667

[35064 rows x 3 columns]


/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_4194/1212030800.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  subset = pd.DataFrame(subset.groupby(["time_tomerge"])["Energy", "Congestion", "Loss"].mean())


In [46]:
print(new_data[ISO]["generation"])

        Unnamed: 0  Unnamed: 0.1                       Time  Dual Fuel  \
0                0             0  2018-01-01 00:05:00-05:00     4619.0   
1                0             0  2022-01-01 00:05:00-05:00     2811.0   
2                0             0  2020-01-01 00:05:00-05:00     2350.0   
3                0             0  2019-01-01 00:05:00-05:00     1872.0   
4                0             0  2021-01-01 00:05:00-05:00     3155.0   
...            ...           ...                        ...        ...   
534719      108039        108039  2022-12-31 23:40:00-05:00     2555.0   
534720      108040        108040  2022-12-31 23:45:00-05:00     2531.0   
534721      108041        108041  2022-12-31 23:50:00-05:00     2511.0   
534722      108042        108042  2022-12-31 23:55:00-05:00     2485.0   
534723      108043        108043  2023-01-01 00:00:00-05:00     2461.0   

         Hydro  Natural Gas  Nuclear  Other Fossil Fuels  Other Renewables  \
0       2607.0       2700.0   541

In [47]:
new_data[ISO]["generation"]["time_tomerge"] = pd.to_datetime(new_data[ISO]["generation"]["Time"], utc=True) -timedelta(hours=5)
new_data[ISO]["generation"] = new_data[ISO]["generation"].sort_values(by="time_tomerge")
print(new_data[ISO]["generation"].columns)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Time', 'Dual Fuel', 'Hydro',
       'Natural Gas', 'Nuclear', 'Other Fossil Fuels', 'Other Renewables',
       'Wind', 'time_tomerge'],
      dtype='object')


In [48]:
subset = new_data[ISO]["generation"]
subset["time_tomerge"] = subset["time_tomerge"].dt.floor("H")
subset = pd.DataFrame(subset.groupby(["time_tomerge"])['Dual Fuel', 'Hydro', 'Natural Gas', 'Nuclear', 'Other Fossil Fuels', 'Other Renewables', 'Wind'].mean())
subset = subset.drop_duplicates().rename(columns={'Dual Fuel': 'gen_dual_fuel', 'Hydro': 'gen_hydro', 'Natural Gas': 'gen_gas', 'Nuclear': 'gen_nuclear', 'Other Fossil Fuels': 'gen_other_fossil', 'Other Renewables': 'gen_other_renewables', 'Wind': 'gen_wind'})
print(subset[subset.index.year !=2018])
generation =subset[subset.index.year !=2018]

                           gen_dual_fuel    gen_hydro      gen_gas  \
time_tomerge                                                         
2019-01-01 00:00:00+00:00    1800.384615  3056.076923  1241.153846   
2019-01-01 01:00:00+00:00    1657.083333  3200.750000  1162.666667   
2019-01-01 02:00:00+00:00    1521.333333  3343.833333  1145.833333   
2019-01-01 03:00:00+00:00    1581.750000  2889.083333  1114.583333   
2019-01-01 04:00:00+00:00    1457.583333  3031.833333  1108.416667   
...                                  ...          ...          ...   
2022-12-31 20:00:00+00:00    3067.500000  3413.083333  3633.333333   
2022-12-31 21:00:00+00:00    2713.083333  3412.750000  3269.500000   
2022-12-31 22:00:00+00:00    2619.416667  3214.916667  2969.583333   
2022-12-31 23:00:00+00:00    2539.000000  2897.333333  2708.583333   
2023-01-01 00:00:00+00:00    2461.000000  2841.000000  2666.000000   

                           gen_nuclear  gen_other_fossil  \
time_tomerge                 

/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_4194/2763448192.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  subset = pd.DataFrame(subset.groupby(["time_tomerge"])['Dual Fuel', 'Hydro', 'Natural Gas', 'Nuclear', 'Other Fossil Fuels', 'Other Renewables', 'Wind'].mean())


In [49]:
new_data[ISO]["load"]["time_tomerge"] = pd.to_datetime(new_data[ISO]["load"]["time_tomerge"], utc=True)
new_data[ISO]["load"] = new_data[ISO]["load"].sort_values(by="time_tomerge")

In [50]:
subset = new_data[ISO]["load"]
subset["time_tomerge"] = subset["time_tomerge"].dt.floor("H")
subset = pd.DataFrame(subset.groupby(["time_tomerge"])['Integrated Load'].mean())
subset = subset.drop_duplicates().rename(columns={'Integrated Load': 'load'})
load = subset
print(load)

                                  load
time_tomerge                          
2019-01-01 00:00:00+00:00  1368.436364
2019-01-01 01:00:00+00:00  1316.454545
2019-01-01 02:00:00+00:00  1266.100000
2019-01-01 03:00:00+00:00  1233.236364
2019-01-01 04:00:00+00:00  1213.927273
...                                ...
2022-12-31 19:00:00+00:00  1549.250155
2022-12-31 20:00:00+00:00  1504.382318
2022-12-31 21:00:00+00:00  1456.384755
2022-12-31 22:00:00+00:00  1403.635418
2022-12-31 23:00:00+00:00  1350.070518

[34375 rows x 1 columns]


In [51]:
print(new_data[ISO]["load"])

        Unnamed: 0           Time Stamp Time Zone    Name   PTID  \
0               10  01/01/2019 00:00:00       EST    WEST  61752   
1                0  01/01/2019 00:00:00       EST  CAPITL  61757   
2                1  01/01/2019 00:00:00       EST  CENTRL  61754   
3                2  01/01/2019 00:00:00       EST  DUNWOD  61760   
4                3  01/01/2019 00:00:00       EST  GENESE  61753   
...            ...                  ...       ...     ...    ...   
385698         263  12/31/2022 23:00:00       EST    WEST  61752   
385699         262  12/31/2022 23:00:00       EST   NORTH  61755   
385700         261  12/31/2022 23:00:00       EST  N.Y.C.  61761   
385701         257  12/31/2022 23:00:00       EST  HUD VL  61758   
385703         260  12/31/2022 23:00:00       EST  MILLWD  61759   

        Integrated Load              time_tomerge  
0             1443.2000 2019-01-01 00:00:00+00:00  
1             1131.1000 2019-01-01 00:00:00+00:00  
2             1597.6000 201

In [52]:
dfs = [energy_rt, energy_da, ancillary_rt, ancillary_da, generation, load]
total = energy_rt
for df in dfs[1:]:
    total = total.merge(df, on="time_tomerge", how="left")
print(total)

                           energy_rt  congestion_rt  losses_rt  energy_da  \
time_tomerge                                                                
2019-01-01 00:00:00+00:00   1.957095     -17.284857   0.056524  10.338000   
2019-01-01 01:00:00+00:00  10.012556      -5.570167   0.275111  10.318000   
2019-01-01 02:00:00+00:00  13.475222      -3.718444   0.370056   9.527333   
2019-01-01 03:00:00+00:00   1.196444      -8.981056   0.036500   9.544000   
2019-01-01 04:00:00+00:00   5.549444      -1.556167   0.156111   9.432667   
...                              ...            ...        ...        ...   
2022-12-31 20:00:00+00:00  30.349611       0.816389   0.801000  32.455333   
2022-12-31 21:00:00+00:00  25.640444       0.538000   0.609333  30.455333   
2022-12-31 22:00:00+00:00  26.697000       0.000389   0.645944  28.338000   
2022-12-31 23:00:00+00:00  25.833222      -0.984333   0.647000  21.406667   
2023-01-01 00:00:00+00:00  26.270667      -5.906000   0.651333        NaN   

In [53]:
print(energy_da.tail(24))

                           energy_da  congestion_da  losses_da
time_tomerge                                                  
2022-12-31 00:00:00+00:00  14.580000     -14.514000   0.418667
2022-12-31 01:00:00+00:00  14.568000     -13.715333   0.396000
2022-12-31 02:00:00+00:00  13.723333     -13.427333   0.349333
2022-12-31 03:00:00+00:00  12.047333     -13.585333   0.280667
2022-12-31 04:00:00+00:00  12.322000     -13.233333   0.312667
2022-12-31 05:00:00+00:00  13.649333     -12.751333   0.356667
2022-12-31 06:00:00+00:00  19.384667     -10.754000   0.508667
2022-12-31 07:00:00+00:00  25.671333      -8.791333   0.640667
2022-12-31 08:00:00+00:00  32.454000      -4.440000   0.841333
2022-12-31 09:00:00+00:00  32.521333      -3.364000   0.900667
2022-12-31 10:00:00+00:00  32.587333      -2.794667   1.009333
2022-12-31 11:00:00+00:00  32.523333      -4.130000   0.900000
2022-12-31 12:00:00+00:00  32.519333      -2.580000   0.940667
2022-12-31 13:00:00+00:00  32.452667      -3.054667   0

In [142]:
total.to_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/NYISO/all_data.csv")

In [143]:
print(ISO_final["NYISO"].columns)
print(total.columns)

Index(['da_10sr', 'da_10nsr', 'da_30or', 'da_regcap', 'rt_10sr', 'rt_10nsr',
       'rt_30or', 'rt_regcap', 'rt_regmov', 'congestion_da', 'congestion_rt',
       'energy_da', 'energy_rt', 'exchange', 'unnamed: 0', 'gas_price',
       'gen_dual_fuel', 'gen_hydro', 'gen_gas', 'gen_nuclear',
       'gen_other_fossil', 'gen_other_renewables', 'gen_wind', 'load',
       'load_forecast', 'losses_da', 'losses_rt'],
      dtype='object')
Index(['energy_rt', 'congestion_rt', 'losses_rt', 'energy_da', 'congestion_da',
       'losses_da', 'rt_10nsr', 'rt_10sr', 'rt_30or', 'rt_regcap', 'rt_regmov',
       'da_10nsr', 'da_10sr', 'da_30or', 'da_regcap', 'gen_dual_fuel',
       'gen_hydro', 'gen_gas', 'gen_nuclear', 'gen_other_fossil',
       'gen_other_renewables', 'gen_wind', 'load'],
      dtype='object')


In [55]:
combined = pd.concat([ISO_final["NYISO"], total])
combined.to_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/ISOs/NYISO.csv")